# Description
Visualization of the filters of VGG16, via gradient ascent in input space.

This script can run on CPU in a few minutes.

Results example: http://i.imgur.com/4nj4KjN.jpg

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from __future__ import print_function

import numpy as np
import time
from keras.preprocessing.image import save_img
from keras.applications import vgg16
from keras import backend as K

# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block5_conv1'

# util function to convert a tensor into a valid image


def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


# build the VGG16 network with ImageNet weights
model = vgg16.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

model.summary()

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())


kept_filters = []
for filter_index in range(200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        width_margin = (img_width + margin) * i
        height_margin = (img_height + margin) * j
        stitched_filters[
            width_margin: width_margin + img_width,
            height_margin: height_margin + img_height, :] = img

# save the result to disk
save_img('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

Using TensorFlow backend.


58892288/58889256 [==============================] - 175s 3us/step
Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_____________________________________________________________

Current loss value: 11.226857
Current loss value: 45.09025
Current loss value: 49.95768
Current loss value: 120.846405
Current loss value: 175.2159
Current loss value: 212.03754
Current loss value: 273.96466
Current loss value: 350.9494
Current loss value: 390.62848
Current loss value: 439.36768
Current loss value: 492.00397
Current loss value: 540.5977
Current loss value: 582.8346
Current loss value: 625.79346
Current loss value: 664.20807
Current loss value: 709.80023
Current loss value: 745.9693
Current loss value: 800.11755
Current loss value: 837.87195
Current loss value: 876.7689
Filter 9 processed in 7s
Processing filter 10
Current loss value: 9.794861
Current loss value: 15.2236395
Current loss value: 42.046684
Current loss value: 58.6384
Current loss value: 109.66994
Current loss value: 151.02899
Current loss value: 189.30911
Current loss value: 224.07571
Current loss value: 268.48193
Current loss value: 306.73132
Current loss value: 346.49957
Current loss value: 386.9244
Curr

Current loss value: 65.38165
Current loss value: 97.621826
Current loss value: 122.44579
Current loss value: 176.40077
Current loss value: 235.3587
Current loss value: 304.81787
Current loss value: 373.3511
Current loss value: 448.42432
Current loss value: 533.08887
Current loss value: 605.5447
Current loss value: 681.134
Current loss value: 755.7605
Current loss value: 830.33044
Current loss value: 906.8587
Current loss value: 980.30695
Current loss value: 1065.2523
Filter 27 processed in 7s
Processing filter 28
Current loss value: 1.2912443
Current loss value: 22.829308
Current loss value: 78.97006
Current loss value: 142.93753
Current loss value: 196.54327
Current loss value: 254.50572
Current loss value: 314.36963
Current loss value: 358.77567
Current loss value: 414.04648
Current loss value: 454.64133
Current loss value: 500.5598
Current loss value: 543.734
Current loss value: 583.25024
Current loss value: 624.67065
Current loss value: 663.85925
Current loss value: 706.1868
Curren

Current loss value: 1210.5004
Filter 40 processed in 7s
Processing filter 41
Current loss value: 0.0
Filter 41 processed in 0s
Processing filter 42
Current loss value: 6.6283207
Current loss value: 61.665943
Current loss value: 146.61932
Current loss value: 248.94153
Current loss value: 354.2629
Current loss value: 434.97324
Current loss value: 516.4832
Current loss value: 585.7545
Current loss value: 674.64404
Current loss value: 749.2516
Current loss value: 813.7995
Current loss value: 891.5516
Current loss value: 952.24854
Current loss value: 1015.864
Current loss value: 1083.1934
Current loss value: 1148.9636
Current loss value: 1195.6892
Current loss value: 1276.0415
Current loss value: 1334.5444
Current loss value: 1403.0209
Filter 42 processed in 8s
Processing filter 43
Current loss value: 7.6836476
Current loss value: 33.50679
Current loss value: 71.15227
Current loss value: 111.67177
Current loss value: 143.00323
Current loss value: 185.1264
Current loss value: 226.38162
Curre

...

Current loss value: 0.0
Filter 129 processed in 1s
Processing filter 130
Current loss value: 0.037124727
Current loss value: 0.0
Filter 130 processed in 1s
Processing filter 131
Current loss value: 5.9480085
Current loss value: 18.131443
Current loss value: 78.804
Current loss value: 146.76138
Current loss value: 211.4758
Current loss value: 275.96832
Current loss value: 340.15186
Current loss value: 390.17297
Current loss value: 449.9203
Current loss value: 504.7902
Current loss value: 545.93134
Current loss value: 595.4718
Current loss value: 650.74365
Current loss value: 694.626
Current loss value: 745.01227
Current loss value: 786.9859
Current loss value: 837.4991
Current loss value: 877.11633
Current loss value: 925.9125
Current loss value: 970.2595
Filter 131 processed in 8s
Processing filter 132
Current loss value: 1.9695911
Current loss value: 29.506413
Current loss value: 66.98939
Current loss value: 103.14703
Current loss value: 136.8003
Current loss value: 174.15974
Current 

Current loss value: 272.61078
Current loss value: 326.10632
Current loss value: 379.42804
Current loss value: 430.34244
Current loss value: 481.4942
Current loss value: 532.1068
Current loss value: 580.11163
Current loss value: 624.33923
Current loss value: 667.5471
Current loss value: 711.42926
Current loss value: 754.1095
Current loss value: 796.28723
Current loss value: 834.10876
Current loss value: 874.6371
Filter 145 processed in 7s
Processing filter 146
Current loss value: 13.004688
Current loss value: 36.481262
Current loss value: 66.39736
Current loss value: 111.05781
Current loss value: 170.72238
Current loss value: 225.26947
Current loss value: 266.09647
Current loss value: 314.89575
Current loss value: 366.15588
Current loss value: 413.95544
Current loss value: 469.23334
Current loss value: 517.3911
Current loss value: 572.89154
Current loss value: 627.48395
Current loss value: 680.2425
Current loss value: 732.83746
Current loss value: 789.947
Current loss value: 845.182
Cur

Current loss value: 517.83435
Current loss value: 547.6577
Current loss value: 576.656
Current loss value: 607.8811
Current loss value: 638.9457
Filter 160 processed in 8s
Processing filter 161
Current loss value: 4.9126835
Current loss value: 35.80932
Current loss value: 81.65148
Current loss value: 134.24533
Current loss value: 173.30518
Current loss value: 218.11842
Current loss value: 257.38358
Current loss value: 301.88916
Current loss value: 342.8785
Current loss value: 382.50366
Current loss value: 422.66132
Current loss value: 461.22507
Current loss value: 493.95517
Current loss value: 530.41876
Current loss value: 565.07874
Current loss value: 595.8136
Current loss value: 634.07025
Current loss value: 662.3364
Current loss value: 701.8588
Current loss value: 733.94775
Filter 161 processed in 8s
Processing filter 162
Current loss value: 0.4107286
Current loss value: 2.3110783
Current loss value: 10.307207
Current loss value: 22.327333
Current loss value: 39.612633
Current loss 

Current loss value: 23.195091
Current loss value: 40.522285
Current loss value: 72.917145
Current loss value: 111.73162
Current loss value: 150.81265
Current loss value: 198.62048
Current loss value: 249.31015
Current loss value: 302.7495
Current loss value: 367.1136
Current loss value: 426.00476
Current loss value: 485.46945
Current loss value: 538.16205
Current loss value: 592.26355
Current loss value: 639.5991
Current loss value: 689.12964
Current loss value: 739.39813
Current loss value: 785.888
Current loss value: 832.92957
Current loss value: 881.5758
Filter 177 processed in 9s
Processing filter 178
Current loss value: 0.0
Filter 178 processed in 1s
Processing filter 179
Current loss value: 0.0
Filter 179 processed in 1s
Processing filter 180
Current loss value: 0.7732923
Current loss value: 11.36418
Current loss value: 34.03703
Current loss value: 59.033764
Current loss value: 81.39446
Current loss value: 103.83827
Current loss value: 134.87003
Current loss value: 155.69777
Curr

Current loss value: 0.0
Filter 197 processed in 2s
Processing filter 198
Current loss value: 0.0
Filter 198 processed in 1s
Processing filter 199
Current loss value: 0.0
Filter 199 processed in 1s
